In [1]:
# load in policy_EIIE model

In [2]:
import sys
import os

script_path = os.path.abspath('')
current_file_dir = os.path.dirname(script_path)
sys.path.insert(0, f"{current_file_dir}/FinRL/")

import logging
import pandas as pd
import numpy as np
import datetime
import torch
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

from finrl.meta.preprocessor.preprocessors import GroupByScaler
from finrl.meta.env_portfolio_optimization.env_portfolio_optimization import PortfolioOptimizationEnv
from finrl.agents.portfolio_optimization.models import DRLAgent
from finrl.agents.portfolio_optimization.architectures import EIIE

import src.env as env 

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'


In [3]:
def load_data() -> pd.DataFrame:
    # download data
    stocks = env.STOCKS
    script_path = os.path.abspath('')
    current_file_dir = os.path.dirname(script_path)
    data_path = os.path.join(
           current_file_dir, 
           "src",
           "data", 
           f"{env.TIME_RANGE_START}-{env.TIME_RANGE_START}-{'-'.join(env.STOCKS)}.pkl"
    )
    if not os.path.isfile(data_path): 
       df = YahooDownloader(
               start_date = env.TIME_RANGE_START,
               end_date = env.TIME_RANGE_END,
               ticker_list = env.STOCKS,
            ).fetch_data()
       df.to_pickle(data_path)
    else:
       df = pd.read_pickle(data_path)
    return df


In [4]:
df = load_data()

# preprocess data, calculate set of technical indicators on the data
fe = FeatureEngineer(
       use_technical_indicator=True,
       tech_indicator_list = env.INDICATORS
    )

df_t = fe.preprocess_data(df)
fmt = "%Y-%m-%d"
train_df = data_split(df, env.TRAIN_START.strftime(fmt), env.TRAIN_END.strftime(fmt))
test_df = data_split(df, env.TEST_START.strftime(fmt), env.TEST_END.strftime(fmt))

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Shape of DataFrame:  (72964, 8)
Successfully added technical indicators


In [5]:
test_df.head(3)

,date,open,high,low,close,volume,tic,day
0,2019-01-02,38.722500,39.712502,38.557499,37.793777,148158800,AAPL,2
0,2019-01-02,192.520004,193.199997,188.949997,162.268097,3009100,AMGN,2
0,2019-01-02,93.910004,96.269997,93.769997,88.524513,4175400,AXP,2


In [6]:
model_kwargs = {
       "lr": env.LR,
       "policy": env.MODEL
}

policy_kwargs = {
       "k_size": 3,
       "time_window": 50
}

In [7]:
environment_train = PortfolioOptimizationEnv(
       train_df,
       initial_amount=env.INITIAL_AMT,
       comission_fee_pct=env.COMISSION_FEE_PCT,
       time_window=env.TIME_WINDOW,
       features=env.FEATURES,
       normalize_df=None
)

In [8]:
model = DRLAgent(environment_train).get_model("pg", device, model_kwargs, policy_kwargs)

In [ ]:
model = DRLAgent.train_model(model, episodes=2)

  0%|                                                                                                                                                                                                                          | 0/2 [00:00<?, ?it/s]

Initial portfolio value:100000
Final portfolio value: 318521.71875
Final accumulative portfolio value: 3.1852171875
Maximum DrawDown: -0.1798230333923052
Sharpe ratio: 1.0037443563667496


 50%|█████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                                                         | 1/2 [00:24<00:24, 24.36s/it]

In [10]:
import copy
model_cpy = copy.deepcopy(model)

In [35]:
environment_test = PortfolioOptimizationEnv(
    test_df,
    initial_amount=env.INITIAL_AMT,
    comission_fee_pct=env.COMISSION_FEE_PCT,
    time_window=env.TIME_WINDOW,
    features=env.FEATURES,
    normalize_df=None
)

In [42]:
# policy = EIIE(time_window=50, device=device)
# policy.load_state_dict(torch.load("policy_EIIE.pt"))

# # 2020
DRLAgent.DRL_validation(model, environment_test, policy=model.train_policy)

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 50 and the array at index 1 has size 0

In [38]:
train_df

,date,open,high,low,close,volume,tic,day
0,2010-01-04,7.622500,7.660714,7.585000,6.461978,493729600,AAPL,0
0,2010-01-04,56.630001,57.869999,56.560001,41.200790,5277400,AMGN,0
0,2010-01-04,40.810001,41.099998,40.389999,33.090431,6894300,AXP,0
0,2010-01-04,55.720001,56.389999,54.799999,43.777546,6186700,BA,0
0,2010-01-04,57.650002,59.189999,57.509998,40.190220,7325600,CAT,0
...,...,...,...,...,...,...,...,...
2263,2018-12-31,248.059998,250.279999,246.940002,230.275452,3123200,UNH,0
2263,2018-12-31,131.679993,132.440002,130.229996,127.043961,7976000,V,0
2263,2018-12-31,55.529999,56.230000,55.070000,42.255074,15331800,VZ,0
2263,2018-12-31,68.540001,68.720001,67.739998,53.132286,4870900,WBA,0


In [39]:
test_df

,date,open,high,low,close,volume,tic,day
0,2019-01-02,38.722500,39.712502,38.557499,37.793785,148158800,AAPL,2
0,2019-01-02,192.520004,193.199997,188.949997,162.268097,3009100,AMGN,2
0,2019-01-02,93.910004,96.269997,93.769997,88.524506,4175400,AXP,2
0,2019-01-02,316.190002,323.950012,313.709991,314.645142,3292200,BA,2
0,2019-01-02,124.029999,127.879997,123.000000,110.966415,4783200,CAT,2
...,...,...,...,...,...,...,...,...
251,2019-12-31,293.660004,294.089996,291.149994,276.337494,2224200,UNH,1
251,2019-12-31,187.059998,188.000000,186.539993,182.088348,5273000,V,1
251,2019-12-31,61.209999,61.400002,60.939999,48.101665,8466700,VZ,1
251,2019-12-31,58.820000,59.470001,58.810001,47.291969,3305100,WBA,1


In [12]:
len(test_df["tic"].unique())

30

In [21]:
for tic in list(test_df["tic"].unique()):
    tic_data = test_df[test_df["tic"] == tic]
    print(len(tic_data))
    print(tic)
    # tic_data = tic_data[..., np.newaxis]
    # state = tic_data if state is None else np.append(state, tic_data, axis=2)


252
AAPL
252
AMGN
252
AXP
252
BA
252
CAT
252
CRM
252
CSCO
252
CVX
252
DIS
252
GS
252
HD
252
HON
252
IBM
252
INTC
252
JNJ
252
JPM
252
KO
252
MCD
252
MMM
252
MRK
252
MSFT
252
NKE
252
PG
252
TRV
252
UNH
252
V
252
VZ
252
WBA
252
WMT
199
DOW


In [23]:
config_tickers.DOW_30_TICKER[:-1]

['AXP',
 'AMGN',
 'AAPL',
 'BA',
 'CAT',
 'CSCO',
 'CVX',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'KO',
 'JPM',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'CRM',
 'VZ',
 'V',
 'WBA',
 'WMT',
 'DIS']